# User based recommendations

In [1]:
import pandas as pd
import scipy
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

In [2]:
df = pd.read_csv('../data/ratings_pivot.csv')

In [3]:
df = df.set_index('Cust_Id')

In [31]:
df

,30,58,77,143,252,257,258,331,341,362,...,17343,17383,17424,17479,17515,17563,17611,17621,17697,17725
Cust_Id,,,,,,,,,,,,,,,,,,,,,
0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,3.0,0.0,5.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,4.0,3.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3.0,0.0,0.0,4.0,4.0,4.0,0.0,5.0,0.0,4.0,...,4.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65529,4.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,3.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
65530,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0
65532,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


### Need to research best way to handle very sparse matrices
- https://machinelearningmastery.com/sparse-matrices-for-machine-learning/
    - "It is computationally expensive to represent and work with sparse matrices as though they are dense, and much improvement in performance can be achieved by using representations and operations that specifically handle the matrix sparsity."
- There are alternative data structures that would be more memory efficient for these type of matrices
- https://pandas.pydata.org/docs/user_guide/sparse.html

### Sparse Data Structure for efficiency

In [4]:
sparse_data = scipy.sparse.csr_matrix(normalize(df.values))

In [15]:
print(sparse_data.toarray())

[[0.0835593  0.         0.         ... 0.         0.         0.        ]
 [0.07265392 0.         0.         ... 0.         0.         0.        ]
 [0.09441212 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.10069467 0.         0.10069467 ... 0.12586834 0.         0.        ]
 [0.13137724 0.         0.         ... 0.16422155 0.         0.        ]
 [0.14210511 0.         0.         ... 0.         0.08526306 0.        ]]


In [12]:
cosine_similarity(sparse_data[0], sparse_data[1])

array([[0.58820381]])

In [24]:
#!pip install sortedcollections

In [49]:
#dictionary for id -> index pairs
id_index_dic = {}
for i, j in enumerate(df.index):
    id_index_dic[j] = i

In [50]:
id_index_dic[5]

3

In [77]:
from sortedcontainers import SortedList
'''
    Get top n similar users based on a given user
    
    params:
        user: user index of interest
        data: user ratings matrix
        n: number of similar users to return 
        
    returns:
        top_scores: 2d list of score and id 
'''
def get_similar_users(user: int, data, n: int):
    top_scores = SortedList(key = lambda x: -x[0])
    for u in range(len(data.toarray())):
        score = cosine_similarity(data[user], data[u])
        top_scores.add([score, u])
    return top_scores[:n+1]
    

In [78]:
%%time
results = get_similar_users(0, sparse_data, 5)

CPU times: user 43.3 s, sys: 96.6 ms, total: 43.4 s
Wall time: 43.4 s


40 second runtime for a single user :(

In [79]:
results

[[array([[1.]]), 0],
 [array([[0.72543739]]), 38299],
 [array([[0.71573383]]), 51704],
 [array([[0.71272392]]), 49760],
 [array([[0.70688714]]), 1303],
 [array([[0.69327405]]), 5284]]

I think creating a local DB and storing this stuff would be better

In [81]:
df.loc[38299].value_counts()

0.0    399
4.0     16
5.0     15
3.0     12
2.0      6
1.0      2
Name: 38299, dtype: int64

Once similar users are retrieved, get their top rated movies